In [1]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, cross_validate, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import make_scorer, f1_score, accuracy_score, precision_score, recall_score

In [45]:
clean_data = pd.read_csv('data/clean_data.csv')
clean_data_gender = pd.read_csv('data/clean_data_gender.csv')
clean_data_alignment = pd.read_csv('data/clean_data_alignment.csv')
clean_data_publisher = pd.read_csv('data/clean_data_publisher.csv')
clean_data_grouped = pd.read_csv('data/clean_data_grouped.csv')
clean_data_grouped_gender = pd.read_csv('data/clean_data_grouped_gender.csv')
clean_data_grouped_alignment = pd.read_csv('data/clean_data_grouped_alignment.csv')
clean_data_grouped_publisher = pd.read_csv('data/clean_data_grouped_publisher.csv')

In [3]:
def get_scoring(average):
    f1 = make_scorer(f1_score, average=average)
    precision = make_scorer(precision_score, average=average)
    recall = make_scorer(recall_score, average=average)
    scoring = {'accuracy': 'accuracy',
           'precision': precision,
           'recall': recall, 
           'f1': f1}
    return scoring

In [4]:
def nested_cross_validation(data, label, clf, grid, average='binary', scalar=True):
    y = data[label]
    X = data.drop(label, axis=1)

    scoring = get_scoring(average)
    f1 = make_scorer(f1_score, average=average)
    
    inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
    outer_cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=1)

    clf = GridSearchCV(estimator=clf, param_grid=grid, cv=inner_cv, scoring=f1, iid=True)
    
    if scalar == True:
        scalar = StandardScaler()
        pipeline = Pipeline([('scalar', scalar), ('clf', clf)])
    else:
        pipeline = Pipeline([('clf', clf)])
    
    nested_scores = cross_validate(estimator=pipeline, X=X, y=y, cv=outer_cv, scoring=scoring)
    return {'accuracy':[nested_scores['test_accuracy'].mean()], 
            'precision': [nested_scores['test_precision'].mean()],
            'recall': [nested_scores['test_recall'].mean()],
            'f1': [nested_scores['test_f1'].mean()],
            'f1_std': [nested_scores['test_accuracy'].std()]}

In [5]:
def cross_validation(data, label, clf, average='binary', scalar=True):
    y = data[label]
    X = data.drop(label, axis=1)

    scoring = get_scoring(average)
    
    if scalar == True:
        scalar = StandardScaler()
        pipeline = Pipeline([('scalar', scalar), ('clf', clf)])
    else:
        pipeline = Pipeline([('clf', clf)])

    scores = cross_validate(estimator=pipeline, X=X, y=y, cv=4, scoring=scoring)
    return {'accuracy':[scores['test_accuracy'].mean()], 
            'precision': [scores['test_precision'].mean()],
            'recall': [scores['test_recall'].mean()],
            'f1': [scores['test_f1'].mean()],
            'f1_std': [scores['test_accuracy'].std()]}

In [40]:
forest_grid = {'max_depth':[2,5,10,15,None]}
forest = RandomForestClassifier(n_estimators=10, random_state=1)

In [41]:
knn_grid = {'n_neighbors':[3, 5, 7, 11]}
knn = KNeighborsClassifier()

In [42]:
logistic_grid = {'C': [0.1, 1]}
logistic = LogisticRegression(penalty='l1', solver='liblinear')

In [43]:
bayes = GaussianNB()

In [10]:
nested_scores = nested_cross_validation(clean_data, "Super Strength", forest, forest_grid)
nested_scores

{'accuracy': [0.8406432748538011],
 'precision': [0.8704726921019056],
 'recall': [0.8306451612903225],
 'f1': [0.8498661127622071],
 'f1_std': [0.025947718346928144]}

In [11]:
nested_scores = nested_cross_validation(clean_data_publisher, "Publisher", knn, knn_grid, average='weighted')
nested_scores

{'accuracy': [0.5363685526260404],
 'precision': [0.5363685526260404],
 'recall': [0.5363685526260404],
 'f1': [0.5363685526260404],
 'f1_std': [0.04445321360643852]}

In [12]:
nested_scores_strength = nested_cross_validation(clean_data_grouped, "Super Strength", logistic, logistic_grid,  scalar=False)
nested_scores_strength

{'accuracy': [0.8435672514619883],
 'precision': [0.8514251437795732],
 'recall': [0.8629032258064516],
 'f1': [0.8570026787234732],
 'f1_std': [0.02424433326283185]}

In [13]:
nested_scores_bayes = cross_validation(clean_data_grouped, "Super Strength", bayes, scalar=False)
nested_scores_bayes

{'accuracy': [0.6608187134502924],
 'precision': [0.7890670011955174],
 'recall': [0.5134408602150538],
 'f1': [0.6217216596201706],
 'f1_std': [0.036285595456113595]}

In [14]:
def append(dict1, dict2):
    if bool(dict1):
        for key in dict1:
            dict1[key].append(dict2[key][0])
        return dict1
    else:
        return dict2.copy()

result = append({}, nested_scores_strength)
result = append(result, nested_scores_bayes)
append(result, nested_scores)

{'accuracy': [0.8435672514619883, 0.6608187134502924, 0.5363685526260404],
 'precision': [0.8514251437795732, 0.7890670011955174, 0.5363685526260404],
 'recall': [0.8629032258064516, 0.5134408602150538, 0.5363685526260404],
 'f1': [0.8570026787234732, 0.6217216596201706, 0.5363685526260404],
 'f1_std': [0.02424433326283185, 0.036285595456113595, 0.04445321360643852]}

In [29]:
def pred_model(data, data_grouped, label, model, grid, scores, average):
    print("default")
    score = cross_validation(data, label, model, average=average)
    scores = append(scores, score)
    print("gridsearch")
    score = nested_cross_validation(data, label, model, grid, average=average)  
    scores = append(scores, score)
    print("agrupado default")
    score = cross_validation(data_grouped, label, model, average=average, scalar=False)
    scores = append(scores, score)
    print("agrupado gridsearch")
    score = nested_cross_validation(data_grouped, label, model, grid, average=average, scalar=False)
    scores = append(scores, score)
    return scores

In [48]:
def pred(data, data_grouped, label, average='binary'):
    scores = {}
    print("CALCULANDO METRICAS PARA RANDOMFOREST")
    scores = pred_model(data, data_grouped, label, forest, forest_grid, scores, average)
    print("CALCULANDO METRICAS PARA KNN")
    scores = pred_model(data, data_grouped, label, knn, knn_grid, scores, average)
    print("CALCULANDO METRICAS PARA LOGISTIC REGRESSION")
    scores = pred_model(data, data_grouped, label, logistic, logistic_grid, scores, average)
    
    print("CALCULANDO METRICAS PARA NAIVE BAYES")
    print("default")
    score = cross_validation(data, label, bayes, average=average)
    scores = append(scores, score)
    print("agrupado")
    score = cross_validation(data_grouped, label, bayes, average=average, scalar=False)
    scores = append(scores, score)
    
    return scores

### Super Strength

In [17]:
index = ['RandomForest_default','RandomForest', 'RandomForest_agrupado_default', 'RandomForest_agrupado', 
         'knn_default', 'knn', 'knn_agrupado_default', 'knn_agrupado', 
         'logistic_default', 'logistic', 'logistic_agrupado_default', 'logistic_agrupado',
         'bayes', 'bayes_agrupado']
scores = pred(clean_data, clean_data_grouped, 'Super Strength')
strength = pd.DataFrame(scores, index=index)
strength

,accuracy,precision,recall,f1,f1_std
RandomForest_default,0.820175,0.856072,0.803763,0.828443,0.041839
RandomForest,0.840643,0.870473,0.830645,0.849866,0.025948
RandomForest_agrupado_default,0.840643,0.868573,0.833333,0.849246,0.042448
RandomForest_agrupado,0.839181,0.867885,0.830645,0.848685,0.026316
knn_default,0.706140,0.775242,0.645161,0.703672,0.055228
knn,0.703216,0.784870,0.626344,0.696211,0.026918
knn_agrupado_default,0.795322,0.931498,0.674731,0.779486,0.037673
knn_agrupado,0.770468,0.886178,0.663978,0.757746,0.019560
logistic_default,0.815789,0.851822,0.801075,0.825072,0.015746
logistic,0.845029,0.880070,0.827957,0.852843,0.018723


### Accelerated Healing

In [32]:
scores = pred(clean_data, clean_data_grouped, 'Accelerated Healing')
healing = pd.DataFrame(scores, index=index)
healing

CALCULANDO METRICAS PARA RANDOMFOREST
default
gridsearch
agrupado default
agrupado gridsearch
CALCULANDO METRICAS PARA KNN
default
gridsearch


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


agrupado default
agrupado gridsearch


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


CALCULANDO METRICAS PARA LOGISTIC REGRESSION
default
gridsearch
agrupado default
agrupado gridsearch
CALCULANDO METRICAS PARA NAIVE BAYES


,accuracy,precision,recall,f1,f1_std
RandomForest_default,0.786581,0.699580,0.356499,0.463991,0.027379
RandomForest,0.791018,0.684817,0.444149,0.527617,0.026361
RandomForest_agrupado_default,0.785067,0.706208,0.356267,0.464115,0.015362
RandomForest_agrupado,0.792292,0.684117,0.431776,0.527574,0.032959
knn_default,0.757374,0.744792,0.151480,0.244949,0.018377
knn,0.766172,0.685764,0.248844,0.364787,0.022093
knn_agrupado_default,0.785161,0.735714,0.319265,0.429399,0.028891
knn_agrupado,0.796669,0.757436,0.366790,0.489706,0.033734
logistic_default,0.742711,0.526906,0.497456,0.509493,0.032649
logistic,0.767582,0.588088,0.481267,0.527302,0.013823


### Stamina

In [19]:
scores = pred(clean_data, clean_data_grouped, 'Stamina')
stamina = pd.DataFrame(scores, index=index)
stamina

,accuracy,precision,recall,f1,f1_std
RandomForest_default,0.783653,0.790609,0.695395,0.739335,0.006157
RandomForest,0.803933,0.800556,0.748246,0.771604,0.028667
RandomForest_agrupado_default,0.776436,0.787758,0.682544,0.728270,0.030918
RandomForest_agrupado,0.786542,0.774714,0.735088,0.752162,0.014592
knn_default,0.732558,0.809063,0.516754,0.629651,0.028046
knn,0.692852,0.692402,0.549518,0.612101,0.022649
knn_agrupado_default,0.779258,0.826665,0.639430,0.717325,0.016906
knn_agrupado,0.777821,0.809827,0.655702,0.722455,0.020905
logistic_default,0.735294,0.711918,0.678860,0.693902,0.020182
logistic,0.798187,0.778803,0.771623,0.772934,0.038540


### Flight

In [20]:
scores = pred(clean_data, clean_data_grouped, 'Flight')
stamina = pd.DataFrame(scores, index=index)
stamina

,accuracy,precision,recall,f1,f1_std
RandomForest_default,0.774927,0.742073,0.451852,0.559555,0.032594
RandomForest,0.801132,0.826363,0.483923,0.601496,0.040078
RandomForest_agrupado_default,0.768976,0.758396,0.419949,0.537096,0.009435
RandomForest_agrupado,0.782109,0.737064,0.506902,0.598521,0.032707
knn_default,0.714967,0.661433,0.237542,0.344228,0.015225
knn,0.688437,0.538317,0.346970,0.418190,0.039283
knn_agrupado_default,0.738376,0.724432,0.287795,0.410113,0.021848
knn_agrupado,0.719208,0.584836,0.438300,0.500689,0.026035
logistic_default,0.747080,0.609122,0.579630,0.592713,0.021951
logistic,0.801209,0.754082,0.566330,0.645776,0.028703


### Gender

In [36]:
scores = pred(clean_data_gender, clean_data_grouped_gender, 'Gender')
gender = pd.DataFrame(scores, index=index)
gender

CALCULANDO METRICAS PARA RANDOMFOREST
default
gridsearch


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-sco

agrupado default
agrupado gridsearch


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-sco

CALCULANDO METRICAS PARA KNN
default
gridsearch
agrupado default
agrupado gridsearch
CALCULANDO METRICAS PARA LOGISTIC REGRESSION
default
gridsearch
agrupado default
agrupado gridsearch


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-sco

CALCULANDO METRICAS PARA NAIVE BAYES


,accuracy,precision,recall,f1,f1_std
RandomForest_default,0.788141,0.704327,0.415909,0.521020,0.008186
RandomForest,0.814596,0.834423,0.426389,0.552129,0.018792
RandomForest_agrupado_default,0.705629,0.443390,0.219444,0.279559,0.015681
RandomForest_agrupado,0.716440,0.464575,0.207197,0.284168,0.030027
knn_default,0.688441,0.364823,0.162879,0.224815,0.012458
knn,0.675970,0.383613,0.269444,0.315983,0.022655
knn_agrupado_default,0.699437,0.430174,0.241793,0.308368,0.022651
knn_agrupado,0.683764,0.417315,0.342045,0.374205,0.039401
logistic_default,0.683802,0.424620,0.404924,0.413361,0.011020
logistic,0.688432,0.433640,0.398737,0.414996,0.014140


### Alignment

In [47]:
scores = pred(clean_data_alignment, clean_data_grouped_alignment, 'Alignment', average='weighted')
alignment = pd.DataFrame(scores, index=index)
alignment

CALCULANDO METRICAS PARA RANDOMFOREST
default


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py

gridsearch


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1

...

agrupado default
agrupado gridsearch


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classificat

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cla

CALCULANDO METRICAS PARA NAIVE BAYES


,accuracy,precision,recall,f1,f1_std
RandomForest_default,0.618595,0.579429,0.618595,0.595176,0.022513
RandomForest,0.636813,0.587254,0.636813,0.600191,0.049082
RandomForest_agrupado_default,0.624740,0.592779,0.624740,0.605447,0.025346
RandomForest_agrupado,0.661448,0.599208,0.661448,0.604236,0.037252
knn_default,0.630903,0.576014,0.630903,0.586917,0.024305
knn,0.617062,0.565790,0.617062,0.585931,0.019837
knn_agrupado_default,0.610889,0.550975,0.610889,0.570444,0.036339
knn_agrupado,0.629304,0.570861,0.629304,0.588680,0.037754
logistic_default,0.606457,0.586119,0.606457,0.594923,0.023731
logistic,0.636944,0.607851,0.636944,0.620218,0.024051


### Publisher

In [46]:
scores = pred(clean_data_publisher, clean_data_grouped_publisher, 'Publisher', average='weighted')
publisher = pd.DataFrame(scores, index=index)
publisher

CALCULANDO METRICAS PARA RANDOMFOREST
default
gridsearch


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:143

agrupado default
agrupado gridsearch


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:143

CALCULANDO METRICAS PARA KNN
default
gridsearch
agrupado default
agrupado gridsearch
CALCULANDO METRICAS PARA LOGISTIC REGRESSION
default


...

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linea

agrupado default
agrupado gridsearch


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linea

CALCULANDO METRICAS PARA NAIVE BAYES


,accuracy,precision,recall,f1,f1_std
RandomForest_default,0.585893,0.574615,0.585893,0.576534,0.023786
RandomForest,0.647424,0.646595,0.647424,0.636761,0.040830
RandomForest_agrupado_default,0.604423,0.598332,0.604423,0.597741,0.034686
RandomForest_agrupado,0.624218,0.613842,0.624218,0.604488,0.047688
knn_default,0.508588,0.566375,0.508588,0.519739,0.034112
knn,0.533223,0.585705,0.533223,0.541911,0.022626
knn_agrupado_default,0.538034,0.571028,0.538034,0.543981,0.038131
knn_agrupado,0.553143,0.584252,0.553143,0.558463,0.042067
logistic_default,0.596611,0.592925,0.596611,0.593588,0.045917
logistic,0.641393,0.638844,0.641393,0.638134,0.032211
